<a href="https://colab.research.google.com/github/Reemaalt/Detection-of-Hallucination-in-Arabic/blob/main/ArabicaQA_Llama_3_1_8B_Answers_Generation_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Llama

In [ ]:
!pip install transformers
!pip install huggingfacehub
!pip install torch

ERROR: Could not find a version that satisfies the requirement huggingfacehub (from versions: none)
ERROR: No matching distribution found for huggingfacehub


In [ ]:
!pip install accelerate

In [ ]:
import os
os.environ['HF_TOKEN']=''
os.environ['HUGGINGFACEHUB_API_TOKEN']=''

## Set the pipeline

##8B

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Load the model and tokenizer
model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float16)
tokenizer = AutoTokenizer.from_pretrained(model_id)

# Move the model to GPU
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

# Create a new pipeline with the quantized model
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, device=0 if device == "cuda" else -1)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

# DATA

## Get ArabiaQA dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

file_path = '/content/drive/MyDrive/test-open.json'
if(file_path):
  print("File found")
else:
  print("File not found")

In [ ]:
import json
import pandas as pd
with open(file_path, 'r', encoding='utf-8') as file:
    data = json.load(file)

df = pd.DataFrame(data)
df.head()

## Generate 10 answers

In [ ]:
def generate_multiple_answers(pipeline, question, num_answers=10):
    answers = []
    for i in range(num_answers):
        temp = 0.1 if i == num_answers - 1 else 1  #sets the temperature for the last answer to be 0.1
        output = pipeline(
            question,
            max_length=100,  #maximum length of the output
            do_sample=True,  #sampling-based text generation, which allows the model
                             #to generate more diverse and creative outputs
            top_k=50,        #top-k sampling
            top_p=0.9,      #nucleus sampling
            temperature=temp  #randomness
        )
        #extracts the generated text
        answers.append(output[0]["generated_answer"])
    return answers

## Create generated answers dataset

In [ ]:
df["generated_answers"] = df["question"].apply(
    lambda q: generate_multiple_answers(pipeline, q, num_answers=10))

print(df.head())

In [ ]:
#save as a JSON file
df.to_json("generated_answers.json", orient="records", indent=4)

#save as a CSV file
df.to_csv("generated_answers.csv", index=False)

In [ ]:
'''#test the pipeline
question = "What is AI?"
print(pipeline(question))